In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

In [2]:
df = pd.read_csv('cleaned_data.csv')

In [3]:
df['Genre'] = df['Genre'].astype('category')
df['Rating'] = df['Rating'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Release_Year  25793 non-null  int64   
 1   Title         25793 non-null  object  
 2   Popularity    25793 non-null  float64 
 3   Vote_Count    25793 non-null  int64   
 4   Vote_Average  25793 non-null  float64 
 5   Genre         25793 non-null  category
 6   Rating        25793 non-null  category
dtypes: category(2), float64(2), int64(2), object(1)
memory usage: 1.0+ MB


The code looks for a specific movie title entered by the user. It then finds similar movies based on popularity, vote count, and average rating The model compares movies using a method that finds the most alike ones. Finally, it shows the original movie and recommends others with similar traits.

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

df_unique = df.drop_duplicates(subset='Title').reset_index(drop=True)

# Prepare features
features = df_unique[['Popularity', 'Vote_Count', 'Vote_Average']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Fit KNN model
knn_model = NearestNeighbors(n_neighbors=6, algorithm='auto')
knn_model.fit(X_scaled)

#input
movie_title = "pursuit".strip().lower()

# Case-insensitive match
df_unique['Title_lower'] = df_unique['Title'].str.lower()

if movie_title in df_unique['Title_lower'].values:
    movie_index = df_unique[df_unique['Title_lower'] == movie_title].index[0]

    # Find similar movies
    distances, indices = knn_model.kneighbors([X_scaled[movie_index]])

    print("\nSelected Movie:", df_unique.loc[movie_index, 'Title'])
    print("Vote Average:", df_unique.loc[movie_index, 'Vote_Average'])

    print("\nRecommended Movies:")
    for idx in indices[0][1:]:  # Skip the first (same movie)
        title = df_unique.loc[idx, 'Title']
        vote_avg = df_unique.loc[idx, 'Vote_Average']
        genre = df_unique.loc[idx, 'Genre']
        print(f"{title} (Rating: {vote_avg}) : {genre}")
else:
    print("Sorry, movie not found.")



Selected Movie: Pursuit
Vote Average: 5.9

Recommended Movies:
Restless (Rating: 5.9) : Action
Kimi (Rating: 6.3) : Thriller
Fistful of Vengeance (Rating: 5.3) : Action
Nightmare Alley (Rating: 7.1) : Crime
My Hero Academia: World Heroes' Mission (Rating: 7.3) : Animation
